In [2]:
import pandas as pd
import numpy as ny

In [3]:
ratings = pd.read_json("../../Dateset/raw/ratings.json", lines=True)
metadata = pd.read_json("../../Dateset/raw/metadata.json", lines=True)
reviews = pd.read_json("../../Dateset/raw/reviews.json", lines=True)
tags = pd.read_json("../../Dateset/raw/tags.json", lines=True)
tag_count = pd.read_json("../../Dateset/raw/tag_count.json", lines=True)
#never used
survey_answer = pd.read_json("../../Dateset/raw/survey_answers.json", lines=True)

In [4]:
metadata.head()

,title,directedBy,starring,dateAdded,avgRating,imdbId,item_id
0,Toy Story (1995),John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",None,3.89146,114709,1
1,Jumanji (1995),Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",None,3.26605,113497,2
2,Grumpier Old Men (1995),Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",None,3.17146,113228,3
3,Waiting to Exhale (1995),Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",None,2.86824,114885,4
4,Father of the Bride Part II (1995),Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",None,3.07620,113041,5


In [5]:
ratings.head()

,item_id,user_id,rating
0,5,997206,3.0
1,10,997206,4.0
2,13,997206,4.0
3,17,997206,5.0
4,21,997206,4.0


In [6]:
tags.head()

,tag,id
0,aardman,22
1,secret service,112
2,hillarious,167
3,christian,270
4,mummy,362


Droping the dateAdded columns

In [7]:
metadata.drop(columns='dateAdded', inplace=True)
metadata['starring'] = metadata['starring'].str.split(",")
metadata.head()

,title,directedBy,starring,avgRating,imdbId,item_id
0,Toy Story (1995),John Lasseter,"[Tim Allen, Tom Hanks, Don Rickles, Jim Var...",3.89146,114709,1
1,Jumanji (1995),Joe Johnston,"[Jonathan Hyde, Bradley Pierce, Robin Willia...",3.26605,113497,2
2,Grumpier Old Men (1995),Howard Deutch,"[Jack Lemmon, Walter Matthau, Ann-Margret , ...",3.17146,113228,3
3,Waiting to Exhale (1995),Forest Whitaker,"[Angela Bassett, Loretta Devine, Whitney Hou...",2.86824,114885,4
4,Father of the Bride Part II (1995),Charles Shyer,"[Steve Martin, Martin Short, Diane Keaton, ...",3.07620,113041,5


In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
ratings["rating"] = scaler.fit_transform(ratings[["rating"]])
ratings.head()

,item_id,user_id,rating
0,5,997206,0.555556
1,10,997206,0.777778
2,13,997206,0.777778
3,17,997206,1.000000
4,21,997206,0.777778


In [9]:
metadata.drop(columns='imdbId', inplace=True)
tags.rename(columns={"id": "tag_id"}, inplace=True)

In [10]:
tag_count = tag_count.merge(tags, on='tag_id', how='left')
tag_count.drop(columns='tag_id', inplace=True)

In [11]:
tag_count.head()

,item_id,num,tag
0,1,4,3d
1,1,1,action
2,1,26,adventure
3,1,1,alien
4,1,21,animated


In [12]:
# Aggregate tags into a dictionary for each item_id
aggregated_df = tag_count.groupby('item_id').apply(
    lambda x: dict(zip(x['tag'], x['num']))
).reset_index(name='tags')

# Resulting DataFrame
print(aggregated_df)

       item_id                                               tags
0            1  {'3d': 4, 'action': 1, 'adventure': 26, 'alien...
1            2  {'adapted from:book': 3, 'adventure': 7, 'anim...
2            3  {'comedy': 1, 'funniest movies': 1, 'funny': 1...
3            4     {'chick flick': 3, 'divorce': 1, 'revenge': 1}
4            5  {'aging': 1, 'comedy': 3, 'family': 4, 'fantas...
...        ...                                                ...
39680   238572                                  {'plot holes': 1}
39681   239006  {'oscar winner': 1, 'propaganda': 1, 'religion...
39682   239040  {'courtroom': 1, 'depression': 1, 'journalism'...
39683   239042  {'black and white': 1, 'no dialogue': 1, 'visu...
39684   239044  {'coming of age': 1, 'jewish': 1, 'jews': 1, '...

[39685 rows x 2 columns]


/var/folders/cb/95mn1h0s29zdm2fkx_n663n40000gn/T/ipykernel_40143/3895900356.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_df = tag_count.groupby('item_id').apply(


In [13]:
ID_title = metadata[['title', 'item_id']]
aggregated_df=aggregated_df.merge(ID_title, on='item_id', how='left')
aggregated_df.head()

,item_id,tags,title
0,1,"{'3d': 4, 'action': 1, 'adventure': 26, 'alien...",Toy Story (1995)
1,2,"{'adapted from:book': 3, 'adventure': 7, 'anim...",Jumanji (1995)
2,3,"{'comedy': 1, 'funniest movies': 1, 'funny': 1...",Grumpier Old Men (1995)
3,4,"{'chick flick': 3, 'divorce': 1, 'revenge': 1}",Waiting to Exhale (1995)
4,5,"{'aging': 1, 'comedy': 3, 'family': 4, 'fantas...",Father of the Bride Part II (1995)


In [14]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["title", "tags"],
    template="Movie name: {title} has the following tags applied by users: {tags}."
)

def create_descriptive_text(row):
    tag_descriptions = ", ".join(
        [f"'{tag}' ({count} times)" for tag, count in row['tags'].items()]
    )
    return prompt_template.format(
        title=row['title'],
        tags=tag_descriptions
    )
aggregated_df['description'] = aggregated_df.apply(create_descriptive_text, axis=1)
aggregated_df.head()

,item_id,tags,title,description
0,1,"{'3d': 4, 'action': 1, 'adventure': 26, 'alien...",Toy Story (1995),Movie name: Toy Story (1995) has the following...
1,2,"{'adapted from:book': 3, 'adventure': 7, 'anim...",Jumanji (1995),Movie name: Jumanji (1995) has the following t...
2,3,"{'comedy': 1, 'funniest movies': 1, 'funny': 1...",Grumpier Old Men (1995),Movie name: Grumpier Old Men (1995) has the fo...
3,4,"{'chick flick': 3, 'divorce': 1, 'revenge': 1}",Waiting to Exhale (1995),Movie name: Waiting to Exhale (1995) has the f...
4,5,"{'aging': 1, 'comedy': 3, 'family': 4, 'fantas...",Father of the Bride Part II (1995),Movie name: Father of the Bride Part II (1995)...


In [15]:
aggregated_df.drop(columns=['tags','title'], inplace=True)

In [16]:
aggregated_df.head() # this one has movies name and the all the tags applied by the users in the description tag.

,item_id,description
0,1,Movie name: Toy Story (1995) has the following...
1,2,Movie name: Jumanji (1995) has the following t...
2,3,Movie name: Grumpier Old Men (1995) has the fo...
3,4,Movie name: Waiting to Exhale (1995) has the f...
4,5,Movie name: Father of the Bride Part II (1995)...


In [17]:
print(metadata.head(), "\n", aggregated_df.head(), "\n", ratings.head(), "\n", reviews.head())

                                title       directedBy  \
0                    Toy Story (1995)    John Lasseter   
1                      Jumanji (1995)     Joe Johnston   
2             Grumpier Old Men (1995)    Howard Deutch   
3            Waiting to Exhale (1995)  Forest Whitaker   
4  Father of the Bride Part II (1995)    Charles Shyer   

                                            starring  avgRating  item_id  
0  [Tim Allen,  Tom Hanks,  Don Rickles,  Jim Var...    3.89146        1  
1  [Jonathan Hyde,  Bradley Pierce,  Robin Willia...    3.26605        2  
2  [Jack Lemmon,  Walter Matthau,  Ann-Margret , ...    3.17146        3  
3  [Angela Bassett,  Loretta Devine,  Whitney Hou...    2.86824        4  
4  [Steve Martin,  Martin Short,  Diane Keaton,  ...    3.07620        5   
    item_id                                        description
0        1  Movie name: Toy Story (1995) has the following...
1        2  Movie name: Jumanji (1995) has the following t...
2        3  M

In [18]:
aggregated_reviews = reviews.groupby('item_id')['txt'].apply(" ".join).reset_index()
aggregated_reviews.rename(columns={'txt': 'all_reviews'}, inplace=True)
aggregated_reviews = aggregated_reviews.merge(ID_title, on='item_id', how='left')

In [35]:
aggregated_reviews.head()

,item_id,all_reviews,title
0,1,The one film that started what is now a cliché...,Toy Story (1995)
1,2,Fun little movie hidden in the back drawer; Mo...,Jumanji (1995)
2,3,"Our Second Visit to Wabasha, Minnesota; Sequel...",Grumpier Old Men (1995)
3,4,Extra in Movie; I was actually an extra in thi...,Waiting to Exhale (1995)
4,5,The Best Movie of All Time; let me tell you.. ...,Father of the Bride Part II (1995)


In [52]:
import ollama
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define the function to summarize reviews using Ollama's LLaMA model with error handling
def summarize_reviews(movie_title, text):
    try:
        prompt = f"Given the following dataset of user reviews for the movie {movie_title}, your task is to analyze and summarize all the reviews provided. Ensure that the summary captures all the essential feedback, sentiments, themes, and details present in each review. Focus on representing the variety of perspectives accurately, including both positive and negative aspects, if present. This summary will help improve personalized movie recommendations, so please maintain a balanced view that reflects the overall impression left by users. The all users reviews: {text}"
        response = ollama.generate(model="llama3.2", prompt=prompt)
        return response.get("response", "Error: Summary not available")
    except Exception as e:
        print(f"Error summarizing {movie_title}: {e}")
        return "Error during summarization"

# Function to process a chunk of DataFrame
def process_chunk(chunk):
    chunk['review_summary'] = chunk.apply(lambda row: summarize_reviews(row['title'], row['all_reviews']), axis=1)
    return chunk

# Function to process DataFrame in batches and apply summarization in parallel
def process_in_batches(df, batch_size=50):
    results = []
    chunks = [df[i:i+batch_size] for i in range(0, df.shape[0], batch_size)]
    
    with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust `max_workers` as needed
        futures = [executor.submit(process_chunk, chunk) for chunk in chunks]
        
        for future in as_completed(futures):
            results.append(future.result())
    
    return pd.concat(results, ignore_index=True)

# Testing on a smaller subset of the DataFrame (e.g., first 100 rows)

testDataset = aggregated_reviews.head(50)
testDataset = process_in_batches(testDataset, batch_size=5)

KeyboardInterrupt: 

,item_id,all_reviews,title
0,1,The one film that started what is now a cliché...,Toy Story (1995)
1,2,Fun little movie hidden in the back drawer; Mo...,Jumanji (1995)
2,3,"Our Second Visit to Wabasha, Minnesota; Sequel...",Grumpier Old Men (1995)
3,4,Extra in Movie; I was actually an extra in thi...,Waiting to Exhale (1995)
4,5,The Best Movie of All Time; let me tell you.. ...,Father of the Bride Part II (1995)


In [ ]:
# from transformers import LlamaForCausalLM, LlamaTokenizer
# import torch
# 
# model_name = "meta-llama/Llama-3.2-11B-Vision-Instruct"  # specify your LLaMA model path or identifier
# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# model = LlamaForCausalLM.from_pretrained(model_name)
# 
# def summarize_reviews(text):
#     # Encode the prompt for summarization
#     prompt = f"Summarize the following movie reviews into a concise summary: {text}"
#     inputs = tokenizer(prompt, return_tensors="pt").to("mps" if torch.cuda.is_available() else "cpu")
#     
#     # Generate the summary
#     summary_ids = model.generate(inputs.input_ids, max_length=100, min_length=30, do_sample=False)
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary
import ollama
import pandas as pd

# Define the function to summarize reviews using ollama's interface
def summarize_reviews(movie_title, text):
    # Create a prompt for the LLaMA model
    prompt = f"Given the following dataset of user reviews for the movie {movie_title}, your task is to analyze and summarize all the reviews provided. Ensure that the summary captures all the essential feedback, sentiments, themes, and details present in each review. Focus on representing the variety of perspectives accurately, including both positive and negative aspects, if present. This summary will help improve personalized movie recommendations, so please maintain a balanced view that reflects the overall impression left by users. The all users reviews :{text}"
    
    # Generate the summary using the LLaMA model through Ollama
    response = ollama.generate(model="llama3.2", prompt=prompt)
    # Extract and return the summary text from the response
    summary = response["response"]
    return summary

#check this thing on the smaller DF before running this function 
aggregated_reviews['review_summary'] = aggregated_reviews.apply(lambda row: summarize_reviews(row['title'], row['all_reviews']),axis=1)



Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1057ee190>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Movie_rec/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
aggregated_reviews.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a function for TF-IDF keyword extraction
def extract_keywords(text, num_keywords=10):
    tfidf = TfidfVectorizer(max_features=num_keywords, stop_words='english')
    tfidf_matrix = tfidf.fit_transform([text])
    keywords = tfidf.get_feature_names_out()
    return ' '.join(keywords)

# Apply keyword extraction to each movie's reviews
aggregated_reviews['keywords'] = aggregated_reviews['all_reviews'].apply(extract_keywords)


In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch
# 
# # Load a sentiment analysis model
# tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
# model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
# 
# def get_sentiment(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
#     outputs = model(**inputs)
#     sentiment = torch.argmax(outputs.logits).item()
#     return sentiment  # 0: negative, 4: positive, etc.
# 
# # For demonstration, get general sentiment for the summary
# aggregated_reviews['sentiment'] = aggregated_reviews['review_summary'].apply(get_sentiment)
